In [1]:
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/adbhut/Desktop/study_material/pii_removal/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
token = "<huggingface token>"
login(token)

In [3]:
model_name = "openai-community/gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
prompt = "Who are you?"
max_new_tokens = 50

In [5]:
tokens = tokenizer(prompt, return_tensors="pt")
tokens

{'input_ids': tensor([[8241,  389,  345,   30]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [ ]:
output = model.generate(
    **tokens, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id
)

In [7]:
completion = tokenizer.decode(output[0], skip_special_tokens=True)

In [8]:
print(completion)

Who are you?

I'm a guy who's been in the business for a long time. I've been in the business for a long time. I've been in the business for a long time. I've been in the business for
